In [16]:
#### initial state######

###### noteeeeee######
##### 1 is the AGENT
##### 2 is the Computerrrrrrrrrr

import numpy as np
row = []
board = []
colSize=7
rowSize=6
for i in range(colSize):
    row.append(0)
for i in range(rowSize):
    board.append(row)

board=np.array(board)

board

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [17]:
##the board will be 2d list the 
#the column should be the column number
# #the player will be Agent or Computer and used to determine the piece color
def is_validMove(Board,Column):
    Column=Column-1
    index=np.where(Board[:,Column]==0)[0]
    if(len(index)==0):
        return False
    else:
        return True


def dropPiece(Board,column,Player):
    isvalid=is_validMove(Board,column)
    column=column-1
    if(isvalid):
        index=np.where(Board[:,column]==0)[0]
        newboard=Board.copy()
        newboard[index[-1],column]=Player
        return newboard,True
    else:
        return None,False



In [18]:
def getAllValidChildren(Board,Player):
    children=[]
    for i in range (colSize):
        child,status=dropPiece(Board,i,Player)
        if(status):
            ##to test the function
            # print(child)
            children.append(child)
    return children


In [19]:
nPP,status=dropPiece(board,4,1)
nPP

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0]])

In [20]:
def check_goal_test(Board, Player):

    for row in range(6):
        for col in range(4):
            if (
                Board[row,col] == Player and
                Board[row,col+1] == Player and
                Board[row,col+2] == Player and
                Board[row,col+3] == Player
            ):
                return Player

    for col in range(7):
        for row in range(3):
            if (
                Board[row,col] == Player and
                Board[row+1,col]== Player and
                Board[row+2,col]== Player and
                Board[row+3,col]== Player
            ):
                return Player

    for row in range(3):
        for col in range(4):
            if (
                Board[row,col] == Player and
                Board[row+1,col+1]== Player and
                Board[row+2,col+2] == Player and
                Board[row+3,col+3] == Player
            ):
                return Player

    for row in range(3):
        for col in range(3, 7):
            if (
                Board[row,col] == Player and
                Board[row+1,col-1]== Player and
                Board[row+2,col-2]== Player and
                Board[row+3,col-3] == Player
            ):
                return Player
    return False



In [21]:
status=check_goal_test(nPP,1)
print(status)


False


In [22]:
def evalLine(line, player, opponent):
    player_count = np.count_nonzero(line==player)
    opponent_count=np.count_nonzero(line==opponent)
    empty_count = np.count_nonzero(line==0)
    if player_count==4 and empty_count==0:
        return 1000
    elif player_count == 3 and empty_count == 1: # If player has 3-in-a-row column/row/diagonal gives heuristic 10
        return 5 
    elif player_count == 2 and empty_count == 2: 
        return 2  
    elif opponent_count == 3 and empty_count == 1:  # If opponent has 3-in-a-row column/row/diagonal gives heuristic -8
        return -4 
    elif opponent_count == 2 and empty_count == 2:
        return -3  
    else:
        return 0

In [23]:
def UtilityOfBoard(Board, Player, Opponent):
    utility = 0
    
    for row in range(rowSize):
        for col in range(colSize-3):
            line = Board[row, col:col+4]
            utility += evalLine(line, Player, Opponent)

    for col in range(colSize):
        for row in range(rowSize-3):
            line = Board[row:row+4,col]
            utility += evalLine(line, Player, Opponent)

    for row in range(rowSize-3):
        for col in range(colSize-3):
            line = [Board[row+i][col+i] for i in range(4)]
            
            utility += evalLine(line, Player, Opponent)

    for row in range(rowSize-3):
        for col in range(3, colSize):
            line = [Board[row+i][col-i] for i in range(4)]
            utility += evalLine(line, Player, Opponent)
    center_col = len(Board[0]) // 2  
    center_count = sum(Board[row][center_col] == Player for row in range(6))
    utility += center_count 

    return utility


In [24]:
def MinimaxAlgorithm(Board,depth,Player):

    opponent=2
    if(Player==1):
        opponent=2
        
    AGentwins=check_goal_test(Board,1)
    if(AGentwins==1):
        return 10000000,Board
    ComputerWins=check_goal_test(Board,2)
    if(ComputerWins==2):
        return -10000000,Board
    if(depth==0):
        return UtilityOfBoard(Board,Player,opponent),Board

    if(Player==1):
        value=float('-inf')
        auxiliaryBoard=None
        children= getAllValidChildren(Board,Player)
        # print (children)
        for child in children:
            print('child board max',child)
            NewVal,_=MinimaxAlgorithm(child, depth-1,2)
            if(NewVal>value):
                auxiliaryBoard=child
                value=NewVal
        return value,auxiliaryBoard
    else:
        value=float('inf')
        auxiliaryBoard=None
        children= getAllValidChildren(Board,Player)
        for child in children:
            print('child board min',child)
            NewVal,_=MinimaxAlgorithm(child,depth-1,1)
            if (NewVal<value):
                auxiliaryBoard=child
                value=NewVal
                print(value)
        return value,auxiliaryBoard

In [46]:
def alphaBetaPruning(Board, depth,Player, alpha=float('-inf'), beta=float('inf')):
    opponent=2
    if(Player==1):
        opponent=2
        
    AGentwins=check_goal_test(Board,1)
    if(AGentwins==1):
        return 10000000+depth,Board
    ComputerWins=check_goal_test(Board,2)
    if(ComputerWins==2):
        return -10000000+depth,Board
    if(depth==0):
        return UtilityOfBoard(Board,Player,opponent),Board
    if(Player==1):
        value=float('-inf')
        auxiliaryBoard=None
        children= getAllValidChildren(Board,Player)
        for child in children:
            # print('child board max',child)
            NewVal,_=alphaBetaPruning(child, depth-1,2,alpha,beta)

            if(NewVal>value):
                auxiliaryBoard=child
                value=NewVal
            if(beta<=value):
                return NewVal,None
            alpha=max(alpha,value)
        return value,auxiliaryBoard
    else:
        value=float('inf')
        auxiliaryBoard=None
        children= getAllValidChildren(Board,Player)
        for child in children:
            # print('child board min',child)
            NewVal,_=alphaBetaPruning(child,depth-1,1,alpha,beta)
            if (NewVal<value):
                auxiliaryBoard=child
                value=NewVal
                # print(value)
            if(alpha>=value):
                return NewVal,None
            beta=min(beta,value)
        return value,auxiliaryBoard
       


In [47]:
newBoard = np.array(
       [[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0]]
        )

In [48]:
Value,nboard=alphaBetaPruning(newBoard,3,1)
print(Value)
print(nboard)

10000000
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0]]


In [49]:
Value,nboard=alphaBetaPruning(nboard,3,1)
print(Value)
print(nboard)

10000002
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 1 1 1 0 0]]


In [ ]:
# newBoard = np.array(
#        [[0, 0, 0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0, 0, 0],
#         [0, 0, 0, 0, 0, 0, 0]]
#         )

       

# MinimaxAlgorithm(newBoard, 6, 1)

In [ ]:
# newBoard = np.array(
#        [[0, 0, 0, 0, 0, 0, 2],
#         [0, 0, 0, 0, 0, 0, 1],
#         [0, 0, 0, 0, 0, 0, 2],
#         [0, 0, 0, 0, 0, 0, 1],
#         [0, 0, 0, 0, 0, 0, 2],
#         [0, 0, 0, 0, 0, 0, 1]]
#         )

# MinimaxAlgorithm(newBoard, 6, 1)

In [ ]:
# newBoard = np.array(
#        [[0, 0, 0, 0, 0, 0, 2],
#         [0, 0, 0, 0, 0, 0, 1],
#         [0, 0, 0, 0, 0, 0, 2],
#         [0, 0, 0, 0, 0, 0, 1],
#         [2, 0, 0, 0, 0, 0, 2],
#         [2, 0, 1, 1, 1, 2, 1]]
#         )
# MinimaxAlgorithm(newBoard, 6, 1)
        